In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

import numpy as np

In [3]:
def get_relative_isotherms(v_range, T_range):
    
    """
    
        This function calculates the p(v, T) plane
        from given a and b parameters of 
        the Van der Waals equation of state for real gases.

    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        T_range: Tuple containing maximum and minimum values of
        temperature to be plotted. Temperatures must be expressed
        in terms of the critical temperature.\n
        v_range: Tuple containing maximum and minimum values of
        volumen to be plotted. Volumens must be expressed
        in exponents of the critical volumen (v in 10^v_range).\n
        
    Returns:
        isotherms: a dictionary containing the values of v and the
        pressures of the isotherms in the following form:\n
            isotherms['v'] = np.array containing the values of v
            in a logarithmic scale.\n
            isotherms['value of the isotherm'] = np.array containing
            the values of p.\
    """

    isotherms = []

    for T in T_range:
        p_R = []
        for v in v_range:
            val = (8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2)
            p_R = np.append(p_R, val)

        isotherms.append(p_R)

    return isotherms

In [4]:
def experimental_isotherms(v_range, T_range):
    
    theo_data = get_relative_isotherms(v_range, T_range)
    expe_data = []
    
    v_limits = []
    p_limits = []

    for i in range(len(theo_data)):
        
        T = T_range[i]

        if T < 1.0:
            
            #try:
            p_theo = theo_data[i]
            p_expe = []
            #p_max = scipy.signal.argrelmax(p)
            #print(p_max)
            #p_min = scipy.signal.argrelmin(p)
            #print(p_min)
            p_range = np.linspace(0.001, 1.0, num=5000)
            p, v_isopressure_limits = find_real_fixed_T(p_range, T)

            for j in range(len(v_range)):
                if v_range[j] > v_isopressure_limits[0] and v_range[j] < v_isopressure_limits[-1]:
                    p_expe.append(p)

                else:
                    p_expe.append(p_theo[j])

            expe_data.append(p_expe)
            v_limits.append([v_isopressure_limits[0], v_isopressure_limits[1]])
            p_limits.append([p, p])

            #except: # Exception as e:

                #expe_data[t] = []
                #v_limits[t] = []
                #p_limits[t] = []
        else:
            expe_data.append(theo_data[i])
            
            if T == 1.0:
                v_limits.append([1.0])
                p_limits.append([1.0])
               
    return expe_data, theo_data, v_limits, p_limits

In [5]:
def get_roots(p, T):
    
    roots = np.roots([1.0, - 1.0/3.0*(1.0 + 8.0*T/p), 3.0/p, -1.0/p])
    roots_in_range = []
    
    for root in roots:
        if np.isreal(root):
            root = np.real(root)
            if root > 0:
                roots_in_range.append(root)
    #print(roots_in_range) 
    roots_in_range.sort()
    
    return roots_in_range

In [6]:
def p_undefined_integral(p_0, v_0, T):
    
    return 8.0/3.0 * T *np.log(v_0 - 1.0/3.0) + 3.0/v_0 - p_0*v_0

In [7]:
def defined_integral(p_0, v_range, T):
    v_0, v_1 = v_range[0], v_range[1]
    return p_undefined_integral(p_0, v_1, T) - p_undefined_integral(p_0, v_0, T)

In [8]:
def find_real_fixed_T(p_range, T):
    
    eps = 1e-3 #1.0/p_range.size
    
    for p in p_range:
        
        roots = get_roots(p, T)
        if len(roots) == 3:
            v_range = (roots[0], roots[2])
            area = defined_integral(p, v_range, T)
            if abs(area) < eps:
                return p, v_range

    return None

In [10]:
def update_tracer(change):
    
    tracer_112_001.visible = True
      
    i = change.get('owner').index
    v = v_values[i]
    p = fig_112_001.marks[0].y[i]

    tracer_112_001.x, tracer_112_001.y = [v], [p]
    
    v_g = v_limits[0][1]
    v_l = v_limits[0][0]
    
    if v > v_limits[0][1]:
        
        bar_112_002.colors = ['#b5e5ff', 'blue']
        bar_112_002.y = [[v],[0.0]]
    
    elif v > v_limits[0][0] and  v < v_limits[0][1]:
        
        x_g, x_l = get_volumes_propotions(v_limits[0], v)
        
        bar_112_002.colors = ['blue', '#b5e5ff']
        bar_112_002.y = [[v_l*x_l], [v_g*x_g]]
        
    elif v < v_limits[0][0]:
        
        bar_112_002.colors = ['#b5e5ff', 'blue']
        bar_112_002.y = [[0.0], [v]]

In [11]:
def get_volumes_propotions(v_limits, v):
    
    #Callen, 239
    
    v_g = v_limits[1]
    v_l = v_limits[0]
    
    x_l = (v_g - v)/(v_g - v_l)
    x_g = (v - v_l)/(v_g - v_l)
    
    return x_g, x_l

In [16]:
v_values = np.linspace(0.4, 5.0, 500)
T_values = [0.95]

expe_data, theo_data, v_limits, p_limits = experimental_isotherms(v_values, T_values)

#######################
#######MIDDLE BLOCK####
#######################

middle_block = widgets.HBox([], layout=widgets.Layout(align_items='center', width='100%'))

scale_x = bqs.LinearScale(min = 0.0, max = max(v_values))
scale_y = bqs.LinearScale(min = 0, max = 2.0)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(0, max(v_values), 5),
                #num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(0, 2.0, 4),
                #num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_112_001 = bq.Figure(title='',
                marks=[],
                axes=[axis_x, axis_y],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=50, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(width='100%', height='500px')
    )

lines_112_001 = bqm.Lines(
                x = v_values, 
                y = expe_data, 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = ['red'],
                #labels = label_values,
)

tracer_112_001 = bqm.Scatter(
        name = '',
        x = [0.0],
        y = [0.0],
        scales = {'x': scale_x, 'y': scale_y}, 
        opacity = [1.0, 0.0],
        visible = False,
        colors = ['#2807a3'],
    )

fig_112_001.marks = [lines_112_001, tracer_112_001]

v_values_rounded = np.round(v_values, 3)

v_slider = widgets.SelectionSlider(
    options=v_values_rounded,
    value=v_values_rounded[-1],
    description=r'\( v \)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout = widgets.Layout(width = '78%', margin='0 0 0 50px')
)

v_slider.observe(update_tracer, 'value')

ordinal_scale = bqs.OrdinalScale()

bar_112_002 = bqm.Bars(x=[1.0],
                       y=[max(v_values)],
                       scales={'x': ordinal_scale, 'y': scale_x},
                       colors=['#b5e5ff', 'blue'])

original_112_002 = bqm.Bars(x=[1.0],
                       y=[max(v_values)],
                       scales={'x': ordinal_scale, 'y': scale_x},
                       colors=['#d9d9d9'],
                       opacities = [0.2])
#axis_x_002 = bqa.Axis(scale=ordinal_scale)
#axis_y_002 = bqa.Axis(scale=y_scale_002,
#                      tick_format='0.2f',
#                      orientation='vertical')

axis_x_002 = bqa.Axis(scale=ordinal_scale,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(0, max(v_values), 5),
                num_ticks=0,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_002 = bqa.Axis(
                scale=scale_x,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(0, max(v_values), 4),
                #num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='v',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_112_002 = bq.Figure(title='',
                marks=[],
                axes=[axis_x_002, axis_y_002],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=50, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(width='100%', height='500px')
    )

fig_112_002.marks = [original_112_002, bar_112_002, ]

middle_block.children = [widgets.VBox([fig_112_001, v_slider], layout = widgets.Layout(align_items='center', width='50%')),
                         widgets.VBox([fig_112_002, ], layout = widgets.Layout(align_items='center', width='50%'))]

#######################
#######MAIN BLOCK######
#######################

main_block = widgets.VBox([], layout=widgets.Layout(align_items='center', width='100%'))

main_block.children = [middle_block]
main_block